In [ ]:
""" 
RemoteRecognizer의 구현 예.


원격 인식기 는 추가 PII 식별 기능을 얻기 위해 
외부 API를 호출 합니다. 이 결과는 다른 인식기에서 수집한 다른 모든 결과에 추가됩니다 .


실제 호출 로직(예: HTTP 또는 gRPC) 
은 이 클래스 내에서 구현되어야 합니다. 
이 예에서는 `requests` 패키지를 사용하여 POST 요청을 수행합니다. 
흐름: 
1. '로드' 중에 'supported_entities'를 호출 
하여 이 서비스가 감지할 수 있는 목록을 가져옵니다. 
2. 'supported_entities' 엔드포인트에서 오는 응답을 번역합니다. 
3. '분석' 중에 PII 감지기에 POST 요청을 수행합니다. 끝점 
4. PII 감지기 끝점에서 오는 응답을 
List[RecognizerResult]로 변환합니다. 
5. 결과 반환


참고: 이 예에서는 Presidio Analyzer 자체를 사용하여 외부 PII 감지기를 모방 합니다.

""

In [ ]:
import json
import logging
from typing import List

In [ ]:
import requests

In [ ]:
from presidio_analyzer import RemoteRecognizer, RecognizerResult
from presidio_analyzer.nlp_engine import NlpArtifacts

In [ ]:
logger = logging.getLogger("presidio-analyzer")

In [ ]:
class ExampleRemoteRecognizer(RemoteRecognizer):
    """
    A reference implementation of a remote recognizer.

    Calls Presidio analyzer as if it was an external remote PII detector
    :param pii_identification_url: Service URL for detecting PII
    :param supported_entities_url: Service URL for getting the supported entities
    by this service
    """

    def __init__(
        self,
        pii_identification_url: str = "https://MYPIISERVICE_URL/detect",
        supported_entities_url: str = "https://MYPIISERVICE_URL/supported_entities",
    ):
        self.pii_identification_url = pii_identification_url
        self.supported_entities_url = supported_entities_url

        super().__init__(
            supported_entities=[], name=None, supported_language="en", version="1.0"
        )

    def load(self) -> None:
        """Call the get_supported_entities API of the external service."""
        try:
            response = requests.get(
                self.supported_entities_url,
                params={"language": self.supported_language},
            )
            self.supported_entities = self._supported_entities_from_response(response)

        except requests.exceptions.RequestException as e:
            logger.error(f"Failed to get supported entities from external service. {e}")
            self.supported_language = []

    def analyze(
        self, text: str, entities: List[str], nlp_artifacts: NlpArtifacts
    ) -> List[RecognizerResult]:
        """Call an external service for PII detection."""

        payload = {"text": text, "language": self.supported_language}

        response = requests.post(
            self.pii_identification_url,
            json=payload,
            timeout=200,
        )

        results = self._recognizer_results_from_response(response)

        return results

    def get_supported_entities(self) -> List[str]:
        """Return the list of supported entities."""
        return self.supported_entities

    @staticmethod
    def _recognizer_results_from_response(
        response: requests.Response,
    ) -> List[RecognizerResult]:
        """Translate the service's response to a list of RecognizerResult."""
        results = json.loads(response.text)
        recognizer_results = [RecognizerResult(**result) for result in results]

        return recognizer_results

    @staticmethod
    def _supported_entities_from_response(response: requests.Response) -> List[str]:
        """Translate the service's supported entities list to Presidio's."""
        return json.loads(response.text)

In [ ]:
if __name__ == "__main__":

    # Illustrative example only: Run Presidio analyzer
    # as if it was an external PII detection mechanism.
    rec = ExampleRemoteRecognizer(
        pii_identification_url="http://localhost:5002/analyze",
        supported_entities_url="http://localhost:5002/supportedentities",
    )

    remote_results = rec.analyze(
        text="My name is Morris", entities=["PERSON"], nlp_artifacts=None
    )
    print(remote_results)